<a href="https://colab.research.google.com/github/Jueunkim429/Cloud-cloud/blob/main/CNN_Cloudcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install libarchive-c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 53.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 84.8 MB/s eta 0:00:00
  Created wheel for cartopy: filename=Cartopy-0.21.1-cp310-cp310-linux_x86_64.whl size=11102748 sha256=c2cb552f9afbc31b367abfe270295c1e2bc6e5d25a5aa79f7dc0948b22c641ea
  Stored in directory: /root/.cache/pip/wheels/30/b0/1a/1c1909e00c76653dc4e2ff48555257c0eb2d1698280c8d9955
Successfully built cartopy


In [4]:
!pip install --upgrade keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from PIL import Image
import os, glob, numpy as np
import os
from sklearn.model_selection import train_test_split
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.backend import set_session as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
from keras.preprocessing.image import ImageDataGenerator
import libarchive
import pydot
import cartopy

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import tensorflow.keras
print(tensorflow.keras.__version__)

2.12.0


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##  ImageDataGenerator 를 활용하여 각 구름 종류별 데이터의 수를 1500개 이상으로 늘리기

In [9]:
!wget https://www.dropbox.com/s/g1acls0cycvn4hg/kaggle_1.zip?dl=1 -O kaggle_1.zip

--2023-06-09 12:17:04--  https://www.dropbox.com/s/g1acls0cycvn4hg/kaggle_1.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/g1acls0cycvn4hg/kaggle_1.zip [following]
--2023-06-09 12:17:04--  https://www.dropbox.com/s/dl/g1acls0cycvn4hg/kaggle_1.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc562e5870dc88868c931b47727d.dl.dropboxusercontent.com/cd/0/get/B9poDs35VNkuaGcdAv5mXGKH__vuf2AT6o2ZQdYYGLVOg8ptG0qUJWpRMiYr_L8ZxSNvQMbuw8QGdZiSuvzvcvbDZhBmbkZok7Qs6UfSbecWvbYSf34_ofiuwm-gCgXRDsOBtfDm5lgxP89UYzpvVeuz4dNInL5QBcLx_prDIBRNLk5zHgeEAgkIvhy3Wtz2_gc/file?dl=1# [following]
--2023-06-09 12:17:04--  https://uc562e5870dc88868c931b47727d.dl.dropboxusercontent.com/cd/0/get/B9poDs35VNkuaGcdAv5mXGKH__vuf2AT6o2ZQdYYGLVOg8ptG0qUJWpRM

In [10]:
!unzip kaggle_1.zip

Archive:  kaggle_1.zip
  inflating: data/submission.csv     
  inflating: data/test/1.jpg         
  inflating: data/test/10.jpg        
  inflating: data/test/100.jpg       
  inflating: data/test/101.jpg       
  inflating: data/test/102.jpg       
  inflating: data/test/103.jpg       
  inflating: data/test/104.jpg       
  inflating: data/test/105.jpg       
  inflating: data/test/106.jpg       
  inflating: data/test/107.jpg       
  inflating: data/test/108.jpg       
  inflating: data/test/109.jpg       
  inflating: data/test/11.jpg        
  inflating: data/test/110.jpg       
  inflating: data/test/111.jpg       
  inflating: data/test/112.jpg       
  inflating: data/test/113.jpg       
  inflating: data/test/114.jpg       
  inflating: data/test/115.jpg       
  inflating: data/test/116.jpg       
  inflating: data/test/117.jpg       
  inflating: data/test/118.jpg       
  inflating: data/test/119.jpg       
  inflating: data/test/12.jpg        
  inflating: data/test/120.

In [11]:
def IDG(fname):
    ImageDG = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        fill_mode='nearest'
    )
    img = tf.keras.preprocessing.image.load_img(fname)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    i = 0
    save = fname.split('/')[0] + "/" + fname.split('/')[1]+ "/" + fname.split('/')[2]  # Modified save path based on the file's directory


    for batch in ImageDG.flow(x, batch_size=1, save_to_dir=save, save_prefix='new',
                              save_format='jpg'):
        i += 1
        if i > 7:
            break


In [12]:
folder_list = os.listdir('data/train')

for f in folder_list:
    fname = "data/train/" + f + "/"
    file_list = os.listdir(fname)
    for i in file_list:
        filename = fname + i
        IDG(filename)


## Image 데이터를 학습 데이터로 변환

In [13]:
img_dir =  "data/train"
categories = os.listdir(img_dir) 
num_classes = len(categories)
 
image_w = 64  #64*64*3 사이즈로 조정
image_h = 64
 
pixel=  image_w * image_h * 3 
X=[]
y=[]
 
for idx, cat in enumerate(categories): # 카테고리를 enumerate를 이용하여 카테고리와 인덱스 사용
    img_dir_detail = img_dir + '/' + cat
    files = glob.glob(img_dir_detail + "/*.jpg")
    for i,f in enumerate(files):
        try:
            img = Image.open(f)
            img = img.convert('RGB')
            img = img.resize((image_w,image_h)) #이미지의 사이즈를 조정
            data = np.asarray(img)
            X.append(data)
            y.append(idx)
            if i % 300 == 0 : # 300번쨰 마다 프린트
                print(cat, " : ", f)
        except:
            print(cat,str(i)," 번째에서 에러")
            
X = np.array(X)  #array로 변환
y = np.array(y)  #array로 변환
 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3) #train test 구분

As  :  data/train/As/new_0_7526.jpg
As  :  data/train/As/new_0_180.jpg
As  :  data/train/As/new_0_4497.jpg
As  :  data/train/As/new_0_6407.jpg
As  :  data/train/As/new_0_4760.jpg
Ct  :  data/train/Ct/new_0_5515.jpg
Ct  :  data/train/Ct/new_0_5265.jpg
Ct  :  data/train/Ct/new_0_6455.jpg
Ct  :  data/train/Ct/new_0_644.jpg
Ct  :  data/train/Ct/new_0_7585.jpg
Ct  :  data/train/Ct/new_0_1654.jpg
Cc  :  data/train/Cc/new_0_6784.jpg
Cc  :  data/train/Cc/new_0_6004.jpg
Cc  :  data/train/Cc/new_0_2855.jpg
Cc  :  data/train/Cc/new_0_774.jpg
Cc  :  data/train/Cc/new_0_1175.jpg
Cc  :  data/train/Cc/new_0_5833.jpg
Cc  :  data/train/Cc/new_0_4957.jpg
St  :  data/train/St/new_0_6784.jpg
St  :  data/train/St/new_0_58.jpg
St  :  data/train/St/new_0_4997.jpg
St  :  data/train/St/St-N165.jpg
St  :  data/train/St/new_0_6275.jpg
St  :  data/train/St/new_0_992.jpg
Cu  :  data/train/Cu/new_0_1152.jpg
Cu  :  data/train/Cu/new_0_8617.jpg
Cu  :  data/train/Cu/new_0_4287.jpg
Cu  :  data/train/Cu/Cu-N069.jpg
Cu  

## 학습데이터 가공

In [14]:
print(X_train.shape) # 데이터 크기 확인
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
 
# img를 array로 변환시 0~255의 값을 가지는데 이것을 0~1로 변환
X_train = X_train.astype(float) / 255.0 
X_test = X_test.astype(float) / 255.0
 
from keras.utils import to_categorical
# 기존의 1의 값을가지는 y값을 [0,1,0,0,---]와 같이 변환
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

(13520, 64, 64, 3)
(13520,)
(5795, 64, 64, 3)
(5795,)


## 모델 구축

In [15]:
image_w = 64
image_h = 64
 
with tf.device('/device:CPU:0'): #가동할 GPU가 없어 CPU로 설정
    model = Sequential() # 모델 선언
  
    # (Convolution layer , Pooling layer) 2개 다중분류여서 activation 은 모두 relu사용
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation="relu")) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25)) #과적합 방지 
              
    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    #FC layer
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    #loss= 다중분류 categorical사용          
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])
    
    model_dir = './model'
    model_path = model_dir + "/cloud_classify.model"
    # 모델결과 저장 및 earlystopping
    checkpoint = ModelCheckpoint(filepath = model_path, monitor='val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 6)


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 64)        0

In [17]:
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_split=0.15, callbacks=[checkpoint, early_stopping])

Epoch 1/100
180/180 [==============================] - ETA: 0s - loss: 2.2404 - accuracy: 0.2074
Epoch 1: val_loss improved from inf to 2.09343, saving model to ./model/cloud_classify.model


180/180 [==============================] - 118s 642ms/step - loss: 2.2404 - accuracy: 0.2074 - val_loss: 2.0934 - val_accuracy: 0.2968
Epoch 2/100
180/180 [==============================] - ETA: 0s - loss: 2.0430 - accuracy: 0.2865
Epoch 2: val_loss improved from 2.09343 to 1.94647, saving model to ./model/cloud_classify.model


180/180 [==============================] - 109s 604ms/step - loss: 2.0430 - accuracy: 0.2865 - val_loss: 1.9465 - val_accuracy: 0.3358
Epoch 3/100
180/180 [==============================] - ETA: 0s - loss: 1.9386 - accuracy: 0.3204
Epoch 3: val_loss improved from 1.94647 to 1.85657, saving model to ./model/cloud_classify.model


180/180 [==============================] - 114s 630ms/step - loss: 1.9386 - accuracy: 0.3204 - val_loss: 1.8566 - val_accuracy: 0.3516
Epoch 4/100
180/180 [==============================] - ETA: 0s - loss: 1.8626 - accuracy: 0.3448
Epoch 4: val_loss improved from 1.85657 to 1.81600, saving model to ./model/cloud_classify.model


180/180 [==============================] - 110s 614ms/step - loss: 1.8626 - accuracy: 0.3448 - val_loss: 1.8160 - val_accuracy: 0.3570
Epoch 5/100
180/180 [==============================] - ETA: 0s - loss: 1.7843 - accuracy: 0.3685
Epoch 5: val_loss improved from 1.81600 to 1.74328, saving model to ./model/cloud_classify.model


...

180/180 [==============================] - 107s 597ms/step - loss: 0.9031 - accuracy: 0.6679 - val_loss: 1.2968 - val_accuracy: 0.5602
Epoch 20/100
180/180 [==============================] - ETA: 0s - loss: 0.8387 - accuracy: 0.6969
Epoch 20: val_loss did not improve from 1.29685
180/180 [==============================] - 108s 599ms/step - loss: 0.8387 - accuracy: 0.6969 - val_loss: 1.3223 - val_accuracy: 0.5621
Epoch 21/100
180/180 [==============================] - ETA: 0s - loss: 0.8121 - accuracy: 0.7022
Epoch 21: val_loss improved from 1.29685 to 1.28085, saving model to ./model/cloud_classify.model


180/180 [==============================] - 113s 626ms/step - loss: 0.8121 - accuracy: 0.7022 - val_loss: 1.2808 - val_accuracy: 0.5685
Epoch 22/100
180/180 [==============================] - ETA: 0s - loss: 0.7747 - accuracy: 0.7124
Epoch 22: val_loss improved from 1.28085 to 1.27841, saving model to ./model/cloud_classify.model


180/180 [==============================] - 107s 594ms/step - loss: 0.7747 - accuracy: 0.7124 - val_loss: 1.2784 - val_accuracy: 0.5774
Epoch 23/100
180/180 [==============================] - ETA: 0s - loss: 0.7514 - accuracy: 0.7251
Epoch 23: val_loss improved from 1.27841 to 1.27476, saving model to ./model/cloud_classify.model


180/180 [==============================] - 109s 608ms/step - loss: 0.7514 - accuracy: 0.7251 - val_loss: 1.2748 - val_accuracy: 0.5878
Epoch 24/100
180/180 [==============================] - ETA: 0s - loss: 0.7194 - accuracy: 0.7339
Epoch 24: val_loss improved from 1.27476 to 1.24661, saving model to ./model/cloud_classify.model


180/180 [==============================] - 113s 626ms/step - loss: 0.7194 - accuracy: 0.7339 - val_loss: 1.2466 - val_accuracy: 0.5952
Epoch 25/100
180/180 [==============================] - ETA: 0s - loss: 0.6724 - accuracy: 0.7477
Epoch 25: val_loss did not improve from 1.24661
180/180 [==============================] - 107s 597ms/step - loss: 0.6724 - accuracy: 0.7477 - val_loss: 1.3212 - val_accuracy: 0.5695
Epoch 26/100
180/180 [==============================] - ETA: 0s - loss: 0.6635 - accuracy: 0.7549
Epoch 26: val_loss did not improve from 1.24661
180/180 [==============================] - 108s 603ms/step - loss: 0.6635 - accuracy: 0.7549 - val_loss: 1.2781 - val_accuracy: 0.5878
Epoch 27/100
180/180 [==============================] - ETA: 0s - loss: 0.6347 - accuracy: 0.7714
Epoch 27: val_loss did not improve from 1.24661
180/180 [==============================] - 113s 623ms/step - loss: 0.6347 - accuracy: 0.7714 - val_loss: 1.3291 - val_accuracy: 0.5814
Epoch 28/100
180/180 [

In [28]:
print("정확도 : %.2f" %(model.evaluate(X_test, y_test)[1]))

182/182 [==============================] - 12s 67ms/step - loss: 1.4557 - accuracy: 0.6352
정확도 : 0.64


### model 2

In [23]:
with tf.device('/device:CPU:0'):
    model2 = Sequential()
  
    model2.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation="relu"))
    model2.add(MaxPooling2D(pool_size=(2,2)))
    model2.add(Dropout(0.25))
              
    model2.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model2.add(MaxPooling2D(pool_size=(2,2)))
    model2.add(Dropout(0.25))
    
    model2.add(Conv2D(128, (3,3), padding="same", activation="relu")) #새로추가
    model2.add(Conv2D(128, (3,3), padding="same", activation="relu")) #새로추가
    model2.add(MaxPooling2D(pool_size=(2,2)))
    model2.add(Dropout(0.25))
    
    model2.add(Flatten())
    model2.add(Dense(256, activation = 'relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(num_classes, activation = 'softmax'))
              
    model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])
    
    model_dir = './model2'
    model_path = model_dir + "/cloud_classify.model2"
    
    checkpoint = ModelCheckpoint(filepath = model_path, monitor='val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 6)


In [24]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 16, 16, 64)       

In [25]:
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_split=0.15, callbacks=[checkpoint, early_stopping])

Epoch 1/100
180/180 [==============================] - ETA: 0s - loss: 0.4549 - accuracy: 0.8283
Epoch 1: val_loss improved from inf to 1.40853, saving model to ./model2/cloud_classify.model2


180/180 [==============================] - 119s 660ms/step - loss: 0.4549 - accuracy: 0.8283 - val_loss: 1.4085 - val_accuracy: 0.6119
Epoch 2/100
180/180 [==============================] - ETA: 0s - loss: 0.4588 - accuracy: 0.8301
Epoch 2: val_loss did not improve from 1.40853
180/180 [==============================] - 123s 684ms/step - loss: 0.4588 - accuracy: 0.8301 - val_loss: 1.4297 - val_accuracy: 0.6119
Epoch 3/100
180/180 [==============================] - ETA: 0s - loss: 0.4350 - accuracy: 0.8399
Epoch 3: val_loss did not improve from 1.40853
180/180 [==============================] - 118s 655ms/step - loss: 0.4350 - accuracy: 0.8399 - val_loss: 1.4183 - val_accuracy: 0.6193
Epoch 4/100
180/180 [==============================] - ETA: 0s - loss: 0.4485 - accuracy: 0.8389
Epoch 4: val_loss did not improve from 1.40853
180/180 [==============================] - 128s 712ms/step - loss: 0.4485 - accuracy: 0.8389 - val_loss: 1.4661 - val_accuracy: 0.6090
Epoch 5/100
180/180 [=======

In [32]:
print("정확도 : %.2f" %(model2.evaluate(X_test, y_test)[1]))

182/182 [==============================] - 27s 148ms/step - loss: 2.3967 - accuracy: 0.0683
정확도 : 0.07


## test